In [1]:
import tensorflow as tf
from glob import glob
import numpy as np
import pickle
import json
import os
import random
import matplotlib.pyplot as plt

In [2]:
# only using CPU
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [3]:
def save_data(output_path, mydata):
    with open(output_path, 'wb') as f:
        
        pickle.dump(mydata, f)
        
def load_data(data_path):
    data = pickle.load(open(data_path, 'rb'))

    return data

def convert_binary_score(severe_score):
    
    if severe_score >= 10:
        return 1
    else:
        return 0
    
def lower_filepath(filepath):
    
    filename, fileformat = filepath.split(".")
    fileformat = fileformat.lower()
    
    converted_filepath = filename + "." + fileformat
    
    return converted_filepath
    
def shuffle_data(mydata):
    mydata = np.array(mydata)
    idx = np.arange(len(mydata))
    random.shuffle(idx)
    
    return mydata[idx]
    
def filter_available_visits(amd_record_visit):
    
    filtered_visits = []
    
    for visit in amd_record_visit:
        re_label_test = False
        re_data_test = False
        le_label_test = False
        le_data_test = False
        
        try:
            this_re_severe_score = visit["AMDSEVRE"]
            if np.isnan(this_re_severe_score):
                re_label_test = False
            else:
                re_label_test = True
        except:
            continue
        
        try:
            this_le_severe_score = visit["AMDSEVLE"]
            if np.isnan(this_le_severe_score):
                le_label_test = False
            else:
                le_label_test = True
        except:
            continue
            
        try:
            this_re_data = visit["RE_IMG"]
            if len(this_re_data) > 0:
                re_data_test = True
            else:
                re_data_test = False
        except:
            continue
            
        try:
            this_le_data = visit["LE_IMG"]
            if len(this_le_data) > 0:
                le_data_test = True
            else:
                le_data_test = False
        except:
            continue
            
        test_result = re_label_test * le_label_test * re_data_test * le_data_test
        
        if test_result == 1:
            filtered_visits.append(visit)
            
    return filtered_visits

In [4]:
def build_patient_dict(data_dir, remove_recurrent=True):
    
    json_data = open(data_dir)
    amd_data = json.load(json_data)
    patient_dict = dict()
    
    count_idx = 0
    count_patient = 0
    count_removed_patient = 0
    for idx, record in enumerate(amd_data):
        count_patient += 1
        if idx % 100 == 0:
            print("{} patients processed...".format(count_idx*100))
            count_idx += 1
        
        this_id = record["ID2"]
        this_visits = record["VISITS"]
        filtered_visits = filter_available_visits(this_visits)
        this_re = dict()
        this_le = dict()
        
        re_year = []
        le_year = []
        re_img = []
        le_img = []
        re_severe_score = []
        le_severe_score = []
        re_late_amd = []
        le_late_amd = []
        
        if len(filtered_visits) <= 1:
            count_removed_patient += 1
            continue
            
        for i, visit in enumerate(filtered_visits):
            
            re_year.append(int(visit["VISNO"])/2)
            le_year.append(int(visit["VISNO"])/2)
            re_img.append(lower_filepath(visit["RE_IMG"]))
            le_img.append(lower_filepath(visit["LE_IMG"]))
            re_severe_score.append(visit["AMDSEVRE"])
            le_severe_score.append(visit["AMDSEVLE"])
            re_late_amd.append(convert_binary_score(visit["AMDSEVRE"]))
            le_late_amd.append(convert_binary_score(visit["AMDSEVLE"]))
            
        re_late_amd = np.array(re_late_amd)
        le_late_amd = np.array(le_late_amd)
        
        this_re["re_year"] = re_year
        this_re["re_img"] = re_img
        this_re["re_severe_score"] = re_severe_score
        this_re["re_late_amd"] = re_late_amd
        
        this_le["le_year"] = le_year
        this_le["le_img"] = le_img
        this_le["le_severe_score"] = le_severe_score
        this_le["le_late_amd"] = le_late_amd
        
        patient_late_amd_check = np.sum(re_late_amd) + np.sum(le_late_amd)
        
        if patient_late_amd_check > 0:
            patient_late_amd_label = 1
        else:
            patient_late_amd_label = 0
        
        patient_dict[this_id] = {"re" : this_re, "le" : this_le, "late_amd_label" : patient_late_amd_label}
    
    print("the number of patients: {}".format(count_patient))
    print("{} patients that have no valid visits were excluded".format(count_removed_patient))
    
    return patient_dict

In [5]:
patient_dict = build_patient_dict("/home/jl5307/current_research/AMD_prediction/data/AREDS_participants_amd3.json", remove_recurrent=True)

0 patients processed...
100 patients processed...
200 patients processed...
300 patients processed...
400 patients processed...
500 patients processed...
600 patients processed...
700 patients processed...
800 patients processed...
900 patients processed...
1000 patients processed...
1100 patients processed...
1200 patients processed...
1300 patients processed...
1400 patients processed...
1500 patients processed...
1600 patients processed...
1700 patients processed...
1800 patients processed...
1900 patients processed...
2000 patients processed...
2100 patients processed...
2200 patients processed...
2300 patients processed...
2400 patients processed...
2500 patients processed...
2600 patients processed...
2700 patients processed...
2800 patients processed...
2900 patients processed...
3000 patients processed...
3100 patients processed...
3200 patients processed...
3300 patients processed...
3400 patients processed...
3500 patients processed...
3600 patients processed...
3700 patients

In [6]:
patient_dict["1001"]

{'re': {'re_year': [0.0, 2.0, 3.0, 4.0, 5.0, 7.0, 8.0, 10.0],
  're_img': ['51685 QUA F2 RE LS.jpg',
   '51685 04 F2 RE LS.jpg',
   '51685 06 F2 RE LS.jpg',
   '51685 08 F2 RE LS.jpg',
   '51685 10 F2 RE LS.jpg',
   '51685 14 F2 RE LS.jpg',
   '51685 16 F2 RE LS.jpg',
   '51685 20 F2 RE LS.jpg'],
  're_severe_score': [4.0, 3.0, 4.0, 4.0, 5.0, 6.0, 8.0, 7.0],
  're_late_amd': array([0, 0, 0, 0, 0, 0, 0, 0])},
 'le': {'le_year': [0.0, 2.0, 3.0, 4.0, 5.0, 7.0, 8.0, 10.0],
  'le_img': ['51685 QUA F2 LE LS.jpg',
   '51685 04 F2 LE LS.jpg',
   '51685 06 F2 LE LS.jpg',
   '51685 08 F2 LE LS.jpg',
   '51685 10 F2 LE LS.jpg',
   '51685 14 F2 LE LS.jpg',
   '51685 16 F2 LE LS.jpg',
   '51685 20 F2 LE LS.jpg'],
  'le_severe_score': [1.0, 3.0, 3.0, 2.0, 4.0, 6.0, 8.0, 6.0],
  'le_late_amd': array([0, 0, 0, 0, 0, 0, 0, 0])},
 'late_amd_label': 0}

In [7]:
# split patient dict into train-validation-test set

In [8]:
def split_patient_dict(patient_dict, train_size, validation_size, test_size):
    
    late_amd_patient_list = []
    non_late_amd_patient_list = []
    
    for pid, value in patient_dict.items():
        
        if value["late_amd_label"] == 1:
            late_amd_patient_list.append(pid)
        else:
            non_late_amd_patient_list.append(pid)
            
    late_amd_patient_list = shuffle_data(late_amd_patient_list)
    non_late_amd_patient_list = shuffle_data(non_late_amd_patient_list)
    
    print("the number of patients have late-AMD at least one eye: {}".format(len(late_amd_patient_list)))
    print("the number of patients do have late-AMD at least one eye: {}".format(len(non_late_amd_patient_list)))
    
    ## train set
    train_late_amd_patient_list = late_amd_patient_list[:int(np.floor(len(late_amd_patient_list)*train_size))]
    train_non_late_amd_patient_list = non_late_amd_patient_list[:int(np.floor(len(non_late_amd_patient_list)*train_size))]  
    
    ## test and validation set
    test_validation_late_amd_patient_list = late_amd_patient_list[int(np.floor(len(late_amd_patient_list)*train_size)):]
    test_validation_non_late_amd_patient_list = non_late_amd_patient_list[int(np.floor(len(non_late_amd_patient_list)*train_size)):]
    
    late_amd_test_validation_slicing_ind = int(np.floor(len(test_validation_late_amd_patient_list) * (test_size / (validation_size+test_size))))
    non_late_amd_test_validation_slicing_ind = int(np.floor(len(test_validation_non_late_amd_patient_list) * (test_size / (validation_size+test_size))))
    
    test_late_amd_patient_list = test_validation_late_amd_patient_list[:late_amd_test_validation_slicing_ind]
    test_non_late_amd_patient_list = test_validation_non_late_amd_patient_list[:non_late_amd_test_validation_slicing_ind]
    
    validation_late_amd_patient_list = test_validation_late_amd_patient_list[late_amd_test_validation_slicing_ind:]
    validation_non_late_amd_patient_list = test_validation_non_late_amd_patient_list[non_late_amd_test_validation_slicing_ind:]

    train_pid_list = []
    validation_pid_list = []
    test_pid_list = []

    train_pid_list.extend(train_late_amd_patient_list)
    train_pid_list.extend(train_non_late_amd_patient_list)
    validation_pid_list.extend(validation_late_amd_patient_list)
    validation_pid_list.extend(validation_non_late_amd_patient_list)
    test_pid_list.extend(test_late_amd_patient_list)
    test_pid_list.extend(test_non_late_amd_patient_list)
    
    train_pid_list = shuffle_data(train_pid_list)
    validation_pid_list = shuffle_data(validation_pid_list)
    test_pid_list = shuffle_data(test_pid_list)

    train_set = dict()
    validation_set = dict()
    test_set = dict()
    
    for pid in train_pid_list:
        train_set[pid] = patient_dict[pid]
    
    for pid in validation_pid_list:
        validation_set[pid] = patient_dict[pid]
        
    for pid in test_pid_list:
        test_set[pid] = patient_dict[pid]
        
    print(len(train_set))
    print(len(validation_set))
    print(len(test_set))
    
    return {"train_set" : train_set, "validation_set" : validation_set, "test_set" : test_set}

In [9]:
splitted_patient_dict = split_patient_dict(patient_dict, 0.7, 0.15, 0.15)

the number of patients have late-AMD at least one eye: 1223
the number of patients do have late-AMD at least one eye: 3092
3020
648
647


In [10]:
save_data("/home/jl5307/current_research/AMD_prediction/img_data/data_dictionary/splitted_patient_dict.pkl", splitted_patient_dict)

In [9]:
splitted_patient_dict = load_data("/home/jl5307/current_research/AMD_prediction/img_data/data_dictionary/splitted_patient_dict.pkl")

In [12]:
splitted_patient_dict["train_set"]["G273"]

{'re': {'re_year': [0.0,
   2.0,
   3.0,
   4.0,
   5.0,
   6.0,
   7.0,
   8.5,
   9.0,
   10.0,
   11.0,
   12.0],
  're_img': ['56516 QUA F2 RE LS.jpg',
   '56516 04 F2 RE LS.jpg',
   '56516 06 F2 RE LS.jpg',
   '56516 08 F2 RE LS.jpg',
   '56516 10 F2 RE LS.jpg',
   '56516 12 F2 RE LS.jpg',
   '56516 14 F2 RE LS.jpg',
   '56516 17 F2 RE LS.jpg',
   '56516 18 F2 RE LS.jpg',
   '56516 20 F2 RE LS.jpg',
   '56516 22 F2 RE LS.jpg',
   '56516 24 F2 RE LS.jpg'],
  're_severe_score': [4.0,
   5.0,
   5.0,
   6.0,
   4.0,
   4.0,
   4.0,
   8.0,
   5.0,
   8.0,
   8.0,
   8.0],
  're_late_amd': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])},
 'le': {'le_year': [0.0,
   2.0,
   3.0,
   4.0,
   5.0,
   6.0,
   7.0,
   8.5,
   9.0,
   10.0,
   11.0,
   12.0],
  'le_img': ['56516 QUA F2 LE LS.jpg',
   '56516 04 F2 LE LS.jpg',
   '56516 06 F2 LE LS.jpg',
   '56516 08 F2 LE LS.jpg',
   '56516 10 F2 LE LS.jpg',
   '56516 12 F2 LE LS.jpg',
   '56516 14 F2 LE LS.jpg',
   '56516 17 F2 LE LS.jpg',
   '

In [30]:
def count_img_year(patient_set_dict):
    
    unique_year = set()
    img_year_list = []
    img_year_dict = dict()
    
    for pid, value in patient_set_dict.items():
        
        re_year = value["re"]["re_year"]
        le_year = value["le"]["le_year"]
        re_late_amd_list = value["re"]["re_late_amd"]
        le_late_amd_list = value["le"]["le_late_amd"]
        
        unique_year.update(set(np.unique(re_year)))
        unique_year.update(set(np.unique(le_year)))
        
        for year, re_label in zip(re_year, re_late_amd_list):
            img_year_list.append((year, re_label))
            
        for year, le_label in zip(re_year, le_late_amd_list):
            img_year_list.append((year, le_label))
            
    for year in unique_year:
        img_year_dict[year] = dict()
        img_year_dict[year][0] = 0
        img_year_dict[year][1] = 0
        
    for (year, label) in img_year_list:
        img_year_dict[year][label] += 1
            
    return img_year_dict

In [31]:
count_img_year(splitted_patient_dict["test_set"])

{0.0: {0: 1030, 1: 84},
 0.5: {0: 26, 1: 24},
 2.0: {0: 953, 1: 119},
 3.0: {0: 1047, 1: 141},
 4.0: {0: 887, 1: 113},
 5.0: {0: 940, 1: 154},
 6.0: {0: 706, 1: 88},
 1.0: {0: 40, 1: 22},
 7.0: {0: 698, 1: 126},
 8.0: {0: 481, 1: 67},
 9.0: {0: 565, 1: 123},
 1.5: {0: 28, 1: 12},
 2.5: {0: 27, 1: 7},
 5.5: {0: 54, 1: 4},
 10.0: {0: 482, 1: 66},
 12.0: {0: 115, 1: 23},
 7.5: {0: 53, 1: 7},
 11.0: {0: 330, 1: 76},
 9.5: {0: 4, 1: 0},
 3.5: {0: 21, 1: 3},
 13.0: {0: 1, 1: 1},
 4.5: {0: 23, 1: 11},
 6.5: {0: 67, 1: 7},
 8.5: {0: 5, 1: 3}}

In [ ]:
# build data dictionary for amd detection task

In [11]:
def build_binary_detection_data_dict(splitted_patient_dict):
    
    train_set = splitted_patient_dict["train_set"]
    validation_set = splitted_patient_dict["validation_set"]
    test_set = splitted_patient_dict["test_set"]
    
    train_eye_dict = dict()
    validation_eye_dict = dict()
    test_eye_dict = dict()
    
    late_amd_img_count = 0
    non_late_amd_img_count = 0
    
    # train set
    for pid, value in train_set.items():
        
        re_img_list = value["re"]["re_img"]
        re_label_list = value["re"]["re_late_amd"]
        
        if len(re_img_list) != len(re_label_list):
            raise ValueError("the length of img_list and label_list must be the same")
        
        le_img_list = value["le"]["le_img"]
        le_label_list = value["le"]["le_late_amd"]
        
        if len(le_img_list) != len(le_label_list):
            raise ValueError("the length of img_list and label_list must be the same")
        
        for img, label in zip(re_img_list, re_label_list):
            if label == 1:
                late_amd_img_count +=1 
            else: 
                non_late_amd_img_count += 1
                
            train_eye_dict[img] = label
        
        for img, label in zip(le_img_list, le_label_list):
            if label == 1:
                late_amd_img_count +=1 
            else: 
                non_late_amd_img_count += 1
            train_eye_dict[img] = label
            
    # validation set
    for pid, value in validation_set.items():
        
        re_img_list = value["re"]["re_img"]
        re_label_list = value["re"]["re_late_amd"]
        
        if len(re_img_list) != len(re_label_list):
            raise ValueError("the length of img_list and label_list must be the same")
        
        le_img_list = value["le"]["le_img"]
        le_label_list = value["le"]["le_late_amd"]
        
        if len(le_img_list) != len(le_label_list):
            raise ValueError("the length of img_list and label_list must be the same")
        
        for img, label in zip(re_img_list, re_label_list):
            validation_eye_dict[img] = label
        
        for img, label in zip(le_img_list, le_label_list):
            validation_eye_dict[img] = label
            
    # test set
    for pid, value in test_set.items():
        
        re_img_list = value["re"]["re_img"]
        re_label_list = value["re"]["re_late_amd"]
        
        if len(re_img_list) != len(re_label_list):
            raise ValueError("the length of img_list and label_list must be the same")
        
        le_img_list = value["le"]["le_img"]
        le_label_list = value["le"]["le_late_amd"]
        
        if len(le_img_list) != len(le_label_list):
            raise ValueError("the length of img_list and label_list must be the same")
        
        for img, label in zip(re_img_list, re_label_list):
            test_eye_dict[img] = label
        
        for img, label in zip(le_img_list, le_label_list):
            test_eye_dict[img] = label
    
    print("late-AMD class ratio: {}".format(late_amd_img_count / (late_amd_img_count+non_late_amd_img_count)))
    print("non late-AMD class ratio: {}".format(non_late_amd_img_count / (late_amd_img_count+non_late_amd_img_count)))
    
    return {"train_set" : train_eye_dict, "validation_set" : validation_eye_dict, "test_set" : test_eye_dict}

In [12]:
binary_detection_data_dict = build_binary_detection_data_dict(splitted_patient_dict)

late-AMD class ratio: 0.12817325307511124
non late-AMD class ratio: 0.8718267469248888


In [13]:
save_data("/home/jl5307/current_research/AMD_prediction/img_data/data_dictionary/binary_detection_data_dict.pkl", binary_detection_data_dict)

In [4]:
binary_detection_data_dict = load_data("/home/jl5307/current_research/AMD_prediction/img_data/data_dictionary/binary_detection_data_dict.pkl")

In [ ]:
# build data dictionary for within 2-year binary prediction 

In [10]:
splitted_patient_dict = load_data("/home/jl5307/current_research/AMD_prediction/img_data/data_dictionary/splitted_patient_dict.pkl")

In [12]:
def count_unique_year(data_set_dict):
    
    unique_year_set = set()
    
    for pid, value in data_set_dict.items():
        re_unique_year = set(np.unique(value["re"]["re_year"]))
        le_unique_year = set(np.unique(value["le"]["le_year"]))
        unique_year_set.update(re_unique_year)
        unique_year_set.update(le_unique_year)
        
    return list(unique_year_set)

def build_binary_prediction_data_dict(splitted_patient_dict, timestamp_delta, remove_recurrent=True):
    
    train_set = splitted_patient_dict["train_set"]
    validation_set = splitted_patient_dict["validation_set"]
    test_set = splitted_patient_dict["test_set"]
    
    train_eye_dict = dict()
    validation_eye_dict = dict()
    test_eye_dict = dict()
    
    # train set
    for pid, value in train_set.items():
        
        re_year = np.array(value["re"]["re_year"])
        re_img_list = value["re"]["re_img"]
        re_severe_score = value["re"]["re_severe_score"]
        re_late_amd = np.array(value["re"]["re_late_amd"])
        
        if len(re_img_list) != len(re_late_amd):
            raise ValueError("the length of img_list and label_list must be the same")
        
        le_year = np.array(value["le"]["le_year"])
        le_img_list = value["le"]["le_img"]
        le_severe_score = value["le"]["le_severe_score"]
        le_late_amd = np.array(value["le"]["le_late_amd"])
        
        if len(le_img_list) != len(le_late_amd):
            raise ValueError("the length of img_list and label_list must be the same")
        
        if remove_recurrent: # remove recurrent late-AMD labels
            
            if np.sum(re_late_amd) > 0: # test whether the eye had late-amd status
                re_first_late_amd_idx = np.where(re_late_amd == 1)[0][0]
                re_year = re_year[:re_first_late_amd_idx+1]
                re_img_list = re_img_list[:re_first_late_amd_idx+1]
                re_severe_score = re_severe_score[:re_first_late_amd_idx+1]
                re_late_amd = re_late_amd[:re_first_late_amd_idx+1]
                
            if np.sum(le_late_amd) > 0: # test whether the eye had late-amd status
                le_first_late_amd_idx = np.where(le_late_amd == 1)[0][0]
                le_year = le_year[:le_first_late_amd_idx+1]
                le_img_list = le_img_list[:le_first_late_amd_idx+1]
                le_severe_score = le_severe_score[:le_first_late_amd_idx+1]
                le_late_amd = le_late_amd[:le_first_late_amd_idx+1]

        if len(re_year) >= 2: 
            # check there are at least more than one 
        
            for idx, year in enumerate(re_year):
                eye_img = re_img_list[idx]
                label_year = year + timestamp_delta
                label_ind1 = np.where(re_year > year, True, False)
                label_ind2 = np.where(re_year <= label_year, True, False)
                label_ind = label_ind1 * label_ind2
            
                if np.sum(label_ind) > 0:
                    train_eye_dict[eye_img] = int(np.max(re_late_amd[label_ind]))
                else:
                    continue
                    
        if len(le_year) >= 2: 
            # check there are at least more than one 
            
            for idx, year in enumerate(le_year):
                eye_img = le_img_list[idx]
                label_year = year + timestamp_delta
                label_ind1 = np.where(le_year > year, True, False)
                label_ind2 = np.where(le_year <= label_year, True, False)
                label_ind = label_ind1 * label_ind2
            
                if np.sum(label_ind) > 0:
                    train_eye_dict[eye_img] = int(np.max(le_late_amd[label_ind]))
                else:
                    continue
                    
    # validation set
    for pid, value in validation_set.items():
        
        re_year = np.array(value["re"]["re_year"])
        re_img_list = value["re"]["re_img"]
        re_severe_score = value["re"]["re_severe_score"]
        re_late_amd = np.array(value["re"]["re_late_amd"])
        
        if len(re_img_list) != len(re_late_amd):
            raise ValueError("the length of img_list and label_list must be the same")
        
        le_year = np.array(value["le"]["le_year"])
        le_img_list = value["le"]["le_img"]
        le_severe_score = value["le"]["le_severe_score"]
        le_late_amd = np.array(value["le"]["le_late_amd"])
        
        if len(le_img_list) != len(le_late_amd):
            raise ValueError("the length of img_list and label_list must be the same")
        
        if remove_recurrent: # remove recurrent late-AMD labels
            
            if np.sum(re_late_amd) > 0: # test whether the eye had late-amd status
                re_first_late_amd_idx = np.where(re_late_amd == 1)[0][0]
                re_year = re_year[:re_first_late_amd_idx+1]
                re_img_list = re_img_list[:re_first_late_amd_idx+1]
                re_severe_score = re_severe_score[:re_first_late_amd_idx+1]
                re_late_amd = re_late_amd[:re_first_late_amd_idx+1]
                
            if np.sum(le_late_amd) > 0: # test whether the eye had late-amd status
                le_first_late_amd_idx = np.where(le_late_amd == 1)[0][0]
                le_year = le_year[:le_first_late_amd_idx+1]
                le_img_list = le_img_list[:le_first_late_amd_idx+1]
                le_severe_score = le_severe_score[:le_first_late_amd_idx+1]
                le_late_amd = le_late_amd[:le_first_late_amd_idx+1]

        if len(re_year) >= 2: 
        
            for idx, year in enumerate(re_year):
                eye_img = re_img_list[idx]
                label_year = year + timestamp_delta
                label_ind1 = np.where(re_year > year, True, False)
                label_ind2 = np.where(re_year <= label_year, True, False)
                label_ind = label_ind1 * label_ind2
            
                if np.sum(label_ind) > 0:
                    validation_eye_dict[eye_img] = int(np.max(re_late_amd[label_ind]))
                else:
                    continue
                    
        if len(le_year) >= 2: 
                
            for idx, year in enumerate(le_year):
                eye_img = le_img_list[idx]
                label_year = year + timestamp_delta
                label_ind1 = np.where(le_year > year, True, False)
                label_ind2 = np.where(le_year <= label_year, True, False)
                label_ind = label_ind1 * label_ind2
            
                if np.sum(label_ind) > 0:
                    validation_eye_dict[eye_img] = int(np.max(le_late_amd[label_ind]))
                else:
                    continue
    
    # test set
    unique_year = count_unique_year(test_set)
    
    for y in unique_year:
        test_eye_dict[y] = dict()

    for pid, value in test_set.items():
        
        re_year = np.array(value["re"]["re_year"])
        re_img_list = value["re"]["re_img"]
        re_severe_score = value["re"]["re_severe_score"]
        re_late_amd = np.array(value["re"]["re_late_amd"])
        
        if len(re_img_list) != len(re_late_amd):
            raise ValueError("the length of img_list and label_list must be the same")
        
        le_year = np.array(value["le"]["le_year"])
        le_img_list = value["le"]["le_img"]
        le_severe_score = value["le"]["le_severe_score"]
        le_late_amd = np.array(value["le"]["le_late_amd"])
        
        if len(le_img_list) != len(le_late_amd):
            raise ValueError("the length of img_list and label_list must be the same")
        
        if remove_recurrent: # remove recurrent late-AMD labels
            
            if np.sum(re_late_amd) > 0: # test whether the eye had late-amd status
                re_first_late_amd_idx = np.where(re_late_amd == 1)[0][0]
                re_year = re_year[:re_first_late_amd_idx+1]
                re_img_list = re_img_list[:re_first_late_amd_idx+1]
                re_severe_score = re_severe_score[:re_first_late_amd_idx+1]
                re_late_amd = re_late_amd[:re_first_late_amd_idx+1]
                
            if np.sum(le_late_amd) > 0: # test whether the eye had late-amd status
                le_first_late_amd_idx = np.where(le_late_amd == 1)[0][0]
                le_year = le_year[:le_first_late_amd_idx+1]
                le_img_list = le_img_list[:le_first_late_amd_idx+1]
                le_severe_score = le_severe_score[:le_first_late_amd_idx+1]
                le_late_amd = le_late_amd[:le_first_late_amd_idx+1]
                
        if len(re_year) >= 2: 
        
            for idx, year in enumerate(re_year):
                eye_img = re_img_list[idx]
                label_year = year + timestamp_delta
                label_ind1 = np.where(re_year > year, True, False)
                label_ind2 = np.where(re_year <= label_year, True, False)
                label_ind = label_ind1 * label_ind2
            
                if np.sum(label_ind) > 0:
                    test_eye_dict[year][eye_img] = int(np.max(re_late_amd[label_ind]))
                else:
                    continue

        if len(le_year) >= 2: 
                
            for idx, year in enumerate(le_year):
                eye_img = le_img_list[idx]
                label_year = year + timestamp_delta
                label_ind1 = np.where(le_year > year, True, False)
                label_ind2 = np.where(le_year <= label_year, True, False)
                label_ind = label_ind1 * label_ind2
            
                if np.sum(label_ind) > 0:
                    test_eye_dict[year][eye_img] = int(np.max(le_late_amd[label_ind]))
                else:
                    continue
                    
    # count the number of data
    print("the number of data in train set: {}".format(len(train_eye_dict)))
    print("the number of data in validation set: {}".format(len(validation_eye_dict)))
    
    test_data_count = 0
    for year, value in test_eye_dict.items():
        test_data_count += len(value)
        
    print("the number of data in test set: {}".format(test_data_count))
          
    return {"train_set" : train_eye_dict, "validation_set" : validation_eye_dict, "test_set" : test_eye_dict}

In [9]:
binary_prediction_data_dict = build_binary_prediction_data_dict(splitted_patient_dict, timestamp_delta=2, remove_recurrent=True)

the number of data in train set: 33870
the number of data in validation set: 7213
the number of data in test set: 7264


In [67]:
save_data("/home/jl5307/current_research/AMD_prediction/img_data/data_dictionary/binary_prediction_data_dict.pkl", binary_prediction_data_dict)

In [18]:
def calculate_class_weight(data_dict):
    
    
    training_set_dict = data_dict["train_set"]
    
    late_amd_count = 0
    non_late_amd_count = 0
    
    for img, label in training_set_dict.items():
        if label == 1:
            late_amd_count += 1
        else:
            non_late_amd_count += 1
            
    total_count = late_amd_count + non_late_amd_count
    
    assert total_count == len(training_set_dict), "data length does not match"
    
    print("class weight for label [0, 1]: [{zero}, {one}]".format(zero=late_amd_count/total_count, one=non_late_amd_count/total_count))

In [19]:
calculate_class_weight(binary_prediction_data_dict)

class weight for label [0, 1]: [0.025214053734868614, 0.9747859462651314]


In [10]:
# build data dictionary for within 2-year longitudinal prediction 

In [40]:
splitted_patient_dict["train_set"]["5535"]["re"]

{'re_year': [0.0, 0.5, 2.0, 3.0, 3.5, 4.0, 5.0, 6.0, 7.0, 9.0, 10.0],
 're_img': ['58682 QUA F2 RE LS.jpg',
  '58682 01 F2 RE LS.jpg',
  '58682 04 F2 RE LS.jpg',
  '58682 06 F2 RE LS.jpg',
  '58682 07 F2 RE LS.jpg',
  '58682 08 F2 RE LS.jpg',
  '58682 10 F2 RE LS.jpg',
  '58682 12 F2 RE LS.jpg',
  '58682 14 F2 RE LS.jpg',
  '58682 18 F2 RE LS.jpg',
  '58682 20 F2 RE LS.jpg'],
 're_severe_score': [7.0,
  6.0,
  6.0,
  11.0,
  11.0,
  11.0,
  11.0,
  11.0,
  11.0,
  11.0,
  11.0],
 're_late_amd': array([0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1])}

In [66]:
def build_longitudinal_prediction_data_dict(splitted_patient_dict, timestamp_delta, remove_recurrent=True, return_statistics=True):
    
    train_set = splitted_patient_dict["train_set"]
    validation_set = splitted_patient_dict["validation_set"]
    test_set = splitted_patient_dict["test_set"]
    
    train_eye_dict = dict()
    validation_eye_dict = dict()
    test_eye_dict = dict()
    
    train_eye_list = []
    train_label_list = []
    validation_eye_list = []
    validation_label_list = []
    
    train_set_year_distribution = []
    validation_set_year_distribution = []
    
    # train set
    for pid, value in train_set.items():
        
        re_year = np.array(value["re"]["re_year"])
        re_img_list = value["re"]["re_img"]
        re_severe_score = value["re"]["re_severe_score"]
        re_late_amd = np.array(value["re"]["re_late_amd"])
        
        if len(re_img_list) != len(re_late_amd):
            raise ValueError("the length of img_list and label_list must be the same")
        
        le_year = np.array(value["le"]["le_year"])
        le_img_list = value["le"]["le_img"]
        le_severe_score = value["le"]["le_severe_score"]
        le_late_amd = np.array(value["le"]["le_late_amd"])
        
        if len(le_img_list) != len(le_late_amd):
            raise ValueError("the length of img_list and label_list must be the same")
        
        if remove_recurrent: # remove recurrent late-AMD labels
            
            if np.sum(re_late_amd) > 0: # test whether the eye had late-amd status
                re_first_late_amd_idx = np.where(re_late_amd == 1)[0][0]
                re_year = re_year[:re_first_late_amd_idx+1]
                re_img_list = re_img_list[:re_first_late_amd_idx+1]
                re_severe_score = re_severe_score[:re_first_late_amd_idx+1]
                re_late_amd = re_late_amd[:re_first_late_amd_idx+1]
                
            if np.sum(le_late_amd) > 0: # test whether the eye had late-amd status
                le_first_late_amd_idx = np.where(le_late_amd == 1)[0][0]
                le_year = le_year[:le_first_late_amd_idx+1]
                le_img_list = le_img_list[:le_first_late_amd_idx+1]
                le_severe_score = le_severe_score[:le_first_late_amd_idx+1]
                le_late_amd = le_late_amd[:le_first_late_amd_idx+1]

        if len(re_year) >= 2: 
        
            for idx, year in enumerate(re_year):
                longitudinal_eye_img_list = re_img_list[:(idx+1)]
                label_year = year + timestamp_delta
                label_ind1 = np.where(re_year > year, True, False)
                label_ind2 = np.where(re_year <= label_year, True, False)
                label_ind = label_ind1 * label_ind2
            
                if np.sum(label_ind) > 0:
                    train_eye_list.append(longitudinal_eye_img_list)
                    train_label_list.append(int(np.max(re_late_amd[label_ind])))
                    train_set_year_distribution.append(year)
                else:
                    continue
                    
        if len(le_year) >= 2: 
                
            for idx, year in enumerate(le_year):
                longitudinal_eye_img_list = le_img_list[:(idx+1)]
                label_year = year + timestamp_delta
                label_ind1 = np.where(le_year > year, True, False)
                label_ind2 = np.where(le_year <= label_year, True, False)
                label_ind = label_ind1 * label_ind2
            
                if np.sum(label_ind) > 0:
                    train_eye_list.append(longitudinal_eye_img_list)
                    train_label_list.append(int(np.max(le_late_amd[label_ind])))
                    train_set_year_distribution.append(year)
                else:
                    continue
                    
    train_eye_dict["eye_list"] = train_eye_list
    train_eye_dict["label_list"] = train_label_list
                    
    # validation set
    for pid, value in validation_set.items():
        
        re_year = np.array(value["re"]["re_year"])
        re_img_list = value["re"]["re_img"]
        re_severe_score = value["re"]["re_severe_score"]
        re_late_amd = np.array(value["re"]["re_late_amd"])
        
        if len(re_img_list) != len(re_late_amd):
            raise ValueError("the length of img_list and label_list must be the same")
        
        le_year = np.array(value["le"]["le_year"])
        le_img_list = value["le"]["le_img"]
        le_severe_score = value["le"]["le_severe_score"]
        le_late_amd = np.array(value["le"]["le_late_amd"])
        
        if len(le_img_list) != len(le_late_amd):
            raise ValueError("the length of img_list and label_list must be the same")
        
        if remove_recurrent: # remove recurrent late-AMD labels
            
            if np.sum(re_late_amd) > 0: # test whether the eye had late-amd status
                re_first_late_amd_idx = np.where(re_late_amd == 1)[0][0]
                re_year = re_year[:re_first_late_amd_idx+1]
                re_img_list = re_img_list[:re_first_late_amd_idx+1]
                re_severe_score = re_severe_score[:re_first_late_amd_idx+1]
                re_late_amd = re_late_amd[:re_first_late_amd_idx+1]
                
            if np.sum(le_late_amd) > 0: # test whether the eye had late-amd status
                le_first_late_amd_idx = np.where(le_late_amd == 1)[0][0]
                le_year = le_year[:le_first_late_amd_idx+1]
                le_img_list = le_img_list[:le_first_late_amd_idx+1]
                le_severe_score = le_severe_score[:le_first_late_amd_idx+1]
                le_late_amd = le_late_amd[:le_first_late_amd_idx+1]

        if len(re_year) >= 2: 
        
            for idx, year in enumerate(re_year):
                longitudinal_eye_img_list = re_img_list[:(idx+1)]
                label_year = year + timestamp_delta
                label_ind1 = np.where(re_year > year, True, False)
                label_ind2 = np.where(re_year <= label_year, True, False)
                label_ind = label_ind1 * label_ind2
            
                if np.sum(label_ind) > 0:
                    validation_eye_list.append(longitudinal_eye_img_list)
                    validation_label_list.append(int(np.max(re_late_amd[label_ind])))
                    validation_set_year_distribution.append(year)
                else:
                    continue
                    
        if len(le_year) >= timestamp_delta: 
                
            for idx, year in enumerate(le_year):
                longitudinal_eye_img_list = le_img_list[:(idx+1)]
                label_year = year + timestamp_delta
                label_ind1 = np.where(le_year > year, True, False)
                label_ind2 = np.where(le_year <= label_year, True, False)
                label_ind = label_ind1 * label_ind2
            
                if np.sum(label_ind) > 0:
                    validation_eye_list.append(longitudinal_eye_img_list)
                    validation_label_list.append(int(np.max(le_late_amd[label_ind])))
                    validation_set_year_distribution.append(year)
                else:
                    continue
                    
    validation_eye_dict["eye_list"] = validation_eye_list
    validation_eye_dict["label_list"] = validation_label_list
    
    # test set
    unique_year = count_unique_year(test_set)
    
    for y in unique_year:
        test_eye_dict[y] = dict()
        test_eye_dict[y]["eye_list"] = []
        test_eye_dict[y]["label_list"] = []

    for pid, value in test_set.items():
        
        re_year = np.array(value["re"]["re_year"])
        re_img_list = value["re"]["re_img"]
        re_severe_score = value["re"]["re_severe_score"]
        re_late_amd = np.array(value["re"]["re_late_amd"])
        
        if len(re_img_list) != len(re_late_amd):
            raise ValueError("the length of img_list and label_list must be the same")
        
        le_year = np.array(value["le"]["le_year"])
        le_img_list = value["le"]["le_img"]
        le_severe_score = value["le"]["le_severe_score"]
        le_late_amd = np.array(value["le"]["le_late_amd"])
        
        if len(le_img_list) != len(le_late_amd):
            raise ValueError("the length of img_list and label_list must be the same")
        
        if remove_recurrent: # remove recurrent late-AMD labels
            
            if np.sum(re_late_amd) > 0: # test whether the eye had late-amd status
                re_first_late_amd_idx = np.where(re_late_amd == 1)[0][0]
                re_year = re_year[:re_first_late_amd_idx+1]
                re_img_list = re_img_list[:re_first_late_amd_idx+1]
                re_severe_score = re_severe_score[:re_first_late_amd_idx+1]
                re_late_amd = re_late_amd[:re_first_late_amd_idx+1]
                
            if np.sum(le_late_amd) > 0: # test whether the eye had late-amd status
                le_first_late_amd_idx = np.where(le_late_amd == 1)[0][0]
                le_year = le_year[:le_first_late_amd_idx+1]
                le_img_list = le_img_list[:le_first_late_amd_idx+1]
                le_severe_score = le_severe_score[:le_first_late_amd_idx+1]
                le_late_amd = le_late_amd[:le_first_late_amd_idx+1]
                
        if len(re_year) >= 2: 
        
            for idx, year in enumerate(re_year):
                longitudinal_eye_img_list = re_img_list[:(idx+1)]
                label_year = year + timestamp_delta
                label_ind1 = np.where(re_year > year, True, False)
                label_ind2 = np.where(re_year <= label_year, True, False)
                label_ind = label_ind1 * label_ind2
            
                if np.sum(label_ind) > 0:
                    test_eye_dict[year]["eye_list"].append(longitudinal_eye_img_list)
                    test_eye_dict[year]["label_list"].append(int(np.max(re_late_amd[label_ind])))
                else:
                    continue

        if len(le_year) >= 2: 
                
            for idx, year in enumerate(le_year):
                longitudinal_eye_img_list = le_img_list[:(idx+1)]
                label_year = year + timestamp_delta
                label_ind1 = np.where(le_year > year, True, False)
                label_ind2 = np.where(le_year <= label_year, True, False)
                label_ind = label_ind1 * label_ind2
            
                if np.sum(label_ind) > 0:
                    test_eye_dict[year]["eye_list"].append(longitudinal_eye_img_list)
                    test_eye_dict[year]["label_list"].append(int(np.max(le_late_amd[label_ind])))
                else:
                    continue
                    
    # count the number of data
    print("the number of data in train set: {}".format(len(train_eye_dict["eye_list"])))
    print("the number of data in validation set: {}".format(len(validation_eye_dict["eye_list"])))
    
    test_data_count = 0
    for year, value in test_eye_dict.items():
        test_data_count += len(value["eye_list"])
        
    print("the number of data in test set: {}".format(test_data_count))
    
    if return_statistics:
        return {"train_set" : train_eye_dict, "validation_set" : validation_eye_dict, "test_set" : test_eye_dict}, train_set_year_distribution, validation_set_year_distribution
    else:
        return {"train_set" : train_eye_dict, "validation_set" : validation_eye_dict, "test_set" : test_eye_dict}

In [67]:
longitudinal_eye_data_dict_timedelta2, ts, vs = build_longitudinal_prediction_data_dict(splitted_patient_dict, timestamp_delta=2, remove_recurrent=True, return_statistics=True)

the number of data in train set: 33870
the number of data in validation set: 7213
the number of data in test set: 7264


In [15]:
save_data("/home/jl5307/current_research/AMD_prediction/img_data/data_dictionary/longitudinal_eye_data_dict_timedelta2.pkl", longitudinal_eye_data_dict_timedelta2)

In [5]:
longitudinal_eye_data_dict_timedelta2 = load_data("/home/jl5307/current_research/AMD_prediction/img_data/data_dictionary/longitudinal_eye_data_dict_timedelta2.pkl")

In [74]:
def count_label_per_year(set_distribution):
    
    count_label_dict = dict()
    unique_year = np.unique(set_distribution)
    
    for year in unique_year:
        count_label_dict[year] = 0
        
    for y in set_distribution:
        count_label_dict[y] += 1
        
    return count_label_dict

In [22]:
def count_test_label_per_year(data_dict):
    
    dataset_dict = data_dict["test_set"]
    unique_year = np.array(list(dataset_dict.keys()))
    unique_year = np.sort(unique_year)
    
    late_amd_count_list = []
    non_late_amd_count_list = []
    
    for year in unique_year:
        
        total_count = len(dataset_dict[year]["label_list"])
        late_amd_count = np.sum(dataset_dict[year]["label_list"])
        non_late_amd_count = total_count - late_amd_count
        
        late_amd_count_list.append(late_amd_count)
        non_late_amd_count_list.append(non_late_amd_count)
        
    print("Year /// late-AMD count /// non late-AMD count")
    for year, late_amd_count, non_late_amd_count in zip(unique_year, late_amd_count_list, non_late_amd_count_list):
        print("Year {}: {} /// {}".format(year, late_amd_count, non_late_amd_count))

In [23]:
count_label_per_year(longitudinal_eye_data_dict_timedelta2)

Year /// late-AMD count /// non late-AMD count
Year 0.0: 41 /// 937
Year 0.5: 2 /// 16
Year 1.0: 3 /// 37
Year 1.5: 5 /// 23
Year 2.0: 26 /// 889
Year 2.5: 1 /// 25
Year 3.0: 29 /// 951
Year 3.5: 1 /// 15
Year 4.0: 19 /// 804
Year 4.5: 1 /// 21
Year 5.0: 23 /// 813
Year 5.5: 1 /// 36
Year 6.0: 15 /// 606
Year 6.5: 0 /// 49
Year 7.0: 11 /// 548
Year 7.5: 0 /// 43
Year 8.0: 6 /// 433
Year 8.5: 0 /// 3
Year 9.0: 5 /// 457
Year 9.5: 1 /// 1
Year 10.0: 5 /// 276
Year 11.0: 1 /// 85
Year 12.0: 0.0 /// 0.0
Year 13.0: 0.0 /// 0.0


In [5]:
longitudinal_eye_data_dict_timedelta2 = load_data("/home/jl5307/current_research/AMD_prediction/img_data/data_dictionary/longitudinal_eye_data_dict_timedelta2.pkl")

In [96]:
def build_n_recent_longitudinal_prediction_data_dict(longitudinal_eye_data_dict, n_recent):
    
    training_eye_list = longitudinal_eye_data_dict["train_set"]["eye_list"]
    training_label_list = longitudinal_eye_data_dict["train_set"]["label_list"]
    validation_eye_list = longitudinal_eye_data_dict["validation_set"]["eye_list"]
    validation_label_list = longitudinal_eye_data_dict["validation_set"]["label_list"]
    test_set = longitudinal_eye_data_dict["test_set"]
    
    n_recent_train_set = dict()
    n_recent_validation_set = dict()
    n_recent_test_set = dict()
    
    n_recent_training_eye_list = []
    n_recent_training_label_list = []
    n_recent_validation_eye_list = []
    n_recent_validation_label_list = []
    
    for eye_list, label in zip(training_eye_list, training_label_list):
        n_recent_training_eye_list.append(eye_list[-n_recent:])
        n_recent_training_label_list.append(label)
        
    n_recent_train_set["eye_list"] = n_recent_training_eye_list
    n_recent_train_set["label_list"] = n_recent_training_label_list
        
    for eye_list, label in zip(validation_eye_list, validation_label_list):
        n_recent_validation_eye_list.append(eye_list[-n_recent:])
        n_recent_validation_label_list.append(label)
        
    n_recent_validation_set["eye_list"] = n_recent_validation_eye_list
    n_recent_validation_set["label_list"] = n_recent_validation_label_list
    
    unique_year = list(test_set.keys())
    
    for year in unique_year:
        this_year_eye_list = test_set[year]["eye_list"]
        this_year_label_list = test_set[year]["label_list"]
        
        this_year_n_recent_test_eye_list = []
        this_year_n_recent_test_label_list = []
        
        for eye_list, label in zip(this_year_eye_list, this_year_label_list):
            this_year_n_recent_test_eye_list.append(eye_list[-n_recent:])
            this_year_n_recent_test_label_list.append(label)
            
        n_recent_test_set[year] = {"eye_list" : this_year_n_recent_test_eye_list, "label_list" : this_year_n_recent_test_label_list}
        
    return {"train_set" : n_recent_train_set, "validation_set" : n_recent_validation_set, "test_set" : n_recent_test_set}

In [97]:
longitudinal_eye_data_dict_timedelta2_recent3 = build_n_recent_longitudinal_prediction_data_dict(longitudinal_eye_data_dict_timedelta2, 3)

In [98]:
save_data("/home/jl5307/current_research/AMD_prediction/img_data/data_dictionary/longitudinal_eye_data_dict_timedelta2_recent3.pkl", longitudinal_eye_data_dict_timedelta2_recent3)

In [81]:
def count_maxlen(dataset):
    
    maxlen = 0
    
    for elem in dataset:
        
        if len(elem) > maxlen:
            maxlen = len(elem)
    
    return maxlen

In [99]:
count_maxlen(longitudinal_eye_data_dict_timedelta2_recent3["validation_set"]["eye_list"])

3

In [100]:
def build_selective_longitudinal_prediction_data_dict(longitudinal_eye_data_dict, n_recent):
    """
    the first and n recent images
    """
    
    training_eye_list = longitudinal_eye_data_dict["train_set"]["eye_list"]
    training_label_list = longitudinal_eye_data_dict["train_set"]["label_list"]
    validation_eye_list = longitudinal_eye_data_dict["validation_set"]["eye_list"]
    validation_label_list = longitudinal_eye_data_dict["validation_set"]["label_list"]
    test_set = longitudinal_eye_data_dict["test_set"]
    
    selective_train_set = dict()
    selective_validation_set = dict()
    selective_test_set = dict()
    
    selective_training_eye_list = []
    selective_training_label_list = []
    selective_validation_eye_list = []
    selective_validation_label_list = []
    
    for eye_list, label in zip(training_eye_list, training_label_list):
        selective_eye_list = []
        
        if len(eye_list) <= (1+n_recent):
            selective_eye_list = eye_list
        else:
            selective_eye_list.append(eye_list[0])
            selective_eye_list.extend(eye_list[-n_recent:])
        
        selective_training_eye_list.append(selective_eye_list)
        selective_training_label_list.append(label)
        
    selective_train_set["eye_list"] = selective_training_eye_list
    selective_train_set["label_list"] = selective_training_label_list
        
    for eye_list, label in zip(validation_eye_list, validation_label_list):
        selective_eye_list = []
        
        if len(eye_list) <= (1+n_recent):
            selective_eye_list = eye_list
        else:
            selective_eye_list.append(eye_list[0])
            selective_eye_list.extend(eye_list[-n_recent:])
        
        selective_validation_eye_list.append(selective_eye_list)
        selective_validation_label_list.append(label)
        
    selective_validation_set["eye_list"] = selective_validation_eye_list
    selective_validation_set["label_list"] = selective_validation_label_list
    
    unique_year = list(test_set.keys())
    
    for year in unique_year:
        this_year_eye_list = test_set[year]["eye_list"]
        this_year_label_list = test_set[year]["label_list"]
        
        this_year_selective_test_eye_list = []
        this_year_selective_test_label_list = []
        
        for eye_list, label in zip(this_year_eye_list, this_year_label_list):
            selective_eye_list = []
            
            if len(eye_list) <= (1+n_recent):
                selective_eye_list = eye_list
            else:
                selective_eye_list.append(eye_list[0])
                selective_eye_list.extend(eye_list[-n_recent:])
                
            this_year_selective_test_eye_list.append(selective_eye_list)
            this_year_selective_test_label_list.append(label)
        
            
        selective_test_set[year] = {"eye_list" : this_year_selective_test_eye_list, "label_list" : this_year_selective_test_label_list}
        
    return {"train_set" : selective_train_set, "validation_set" : selective_validation_set, "test_set" : selective_test_set}

In [101]:
longitudinal_eye_data_dict_timedelta2_selective = build_selective_longitudinal_prediction_data_dict(longitudinal_eye_data_dict_timedelta2, 2)

In [102]:
save_data("/home/jl5307/current_research/AMD_prediction/img_data/data_dictionary/longitudinal_eye_data_dict_timedelta2_selective.pkl", longitudinal_eye_data_dict_timedelta2_selective)

In [103]:
longitudinal_eye_data_dict_timedelta2_selective["train_set"]["label_list"]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
